In [1]:
from IPython.core.display import HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%config IPCompleter.use_jedi=False

# Tokenizing

In [4]:
!../mosesdecoder/scripts/tokenizer/tokenizer.perl -l en -threads 20 < ../data/raw/europarl-v7/fr/europarl-v7.fr-en.en > ../data/processed/europarl-v7/euro.tok.en
!../mosesdecoder/scripts/tokenizer/tokenizer.perl -l fr -threads 20 < ../data/raw/europarl-v7/fr/europarl-v7.fr-en.fr > ../data/processed/europarl-v7/euro.tok.fr

Tokenizer Version 1.1
Language: en
Number of threads: 20
Tokenizer Version 1.1
Language: fr
Number of threads: 20


# True casing

In [5]:
!../mosesdecoder/scripts/recaser/train-truecaser.perl --model ../models/truecaser/euro.model.en --corpus ../data/processed/europarl-v7/euro.tok.en
!../mosesdecoder/scripts/recaser/train-truecaser.perl --model ../models/truecaser/euro.model.fr --corpus ../data/processed/europarl-v7/euro.tok.fr

In [6]:
!../mosesdecoder/scripts/recaser/truecase.perl --model ../models/truecaser/euro.model.en < ../data/processed/europarl-v7/euro.tok.en > ../data/processed/europarl-v7/euro.true.en
!../mosesdecoder/scripts/recaser/truecase.perl --model ../models/truecaser/euro.model.fr < ../data/processed/europarl-v7/euro.tok.fr > ../data/processed/europarl-v7/euro.true.fr

# Cleaning

In [6]:
!../mosesdecoder/scripts/training/clean-corpus-n.perl ../data/processed/europarl-v7/euro.true fr en ../data/processed/europarl-v7/euro.clean 1 20

clean-corpus.perl: processing ../data/processed/europarl-v7/euro.true.fr & .en to ../data/processed/europarl-v7/euro.clean, cutoff 1-20, ratio 9
..........(100000)..........(200000)..........(300000)..........(400000)..........(500000)..........(600000)..........(700000)..........(800000)..........(900000)..........(1000000)..........(1100000)..........(1200000)..........(1300000)..........(1400000)..........(1500000)..........(1600000)..........(1700000)..........(1800000)..........(1900000)..........(2000000)
Input sentences: 2007723  Output sentences:  604809


# Language Modeling

In [2]:
%%time
!../mosesdecoder/bin/lmplz -o 6 -S 65% < ../data/processed/europarl-v7/euro.true.en > ../models/lm/euro.en.arpa
!../mosesdecoder/bin/lmplz -o 6 -S 65% < ../data/processed/europarl-v7/euro.true.fr > ../models/lm/euro.fr.arpa

=== 1/5 Counting and sorting n-grams ===
Reading /tf/data/processed/europarl-v7/euro.true.en
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
tcmalloc: large alloc 6894206976 bytes == 0x55e6050c0000 @ 
tcmalloc: large alloc 36769071104 bytes == 0x55e79ff94000 @ 
****************************************************************************************************
Unigram tokens 55732029 types 128090
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:1537080 2:2688424960 3:5040796672 4:8065274880 5:11761859584 6:16130549760
tcmalloc: large alloc 16130555904 bytes == 0x55e6050c0000 @ 
tcmalloc: large alloc 2688425984 bytes == 0x55e9c698c000 @ 
tcmalloc: large alloc 5040799744 bytes == 0x55ea66d6e000 @ 
tcmalloc: large alloc 8065277952 bytes == 0x55eb934b6000 @ 
tcmalloc: large alloc 11761860608 bytes == 0x55f0321f8000 @ 
Statistics:
1 128090 D1=0.644989 D2=1.03315 D3+=1.36974
2 3093712 D1=0.695945 D2=1.0676 D3+=1.39135
3 1

In [3]:
!../mosesdecoder/bin/build_binary ../models/lm/euro.en.arpa ../models/lm/euro.en.blm
!../mosesdecoder/bin/build_binary ../models/lm/euro.fr.arpa ../models/lm/euro.fr.blm

Reading ../models/lm/euro.en.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS
Reading ../models/lm/euro.fr.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS


# Parsing

In [7]:
%%time
!java -jar ../berkeleyparser/BerkeleyParser-1.6.jar -gr ../berkeleyparser/eng_sm6.gr -inputFile ../data/processed/europarl-v7/euro.clean.en | ../mosesdecoder/scripts/training/wrappers/berkeleyparsed2mosesxml.perl > ../data/processed/europarl-v7/euro.syntax.en

The score is -Infinity and the state is supposed to be ROOT
The insideScores are [0.018813411407087408] and the outsideScores are [1.0]
The maxcScore is -Infinity
^C
CPU times: user 8.84 s, sys: 1.86 s, total: 10.7 s
Wall time: 6min 38s
